In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
!nvidia-smi

Fri Aug 14 08:49:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:37:00.0 Off |                  Off |
| 31%   43C    P2    35W / 250W |   1673MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
import nfp

In [4]:
from preprocess_inputs import preprocessor
preprocessor.from_json('tfrecords/preprocessor.json')

from loss import AtomInfMask, KLWithLogits

model = tf.keras.models.load_model(
    '20200812_kl_divergence_faster_lr/best_model.hdf5',
    custom_objects={**nfp.custom_objects,
                    **{'AtomInfMask': AtomInfMask, 'KLWithLogits': KLWithLogits}})

In [5]:
def parse_example(example):
    parsed = tf.io.parse_single_example(example, features={
        **preprocessor.tfrecord_features,
        **{'spin': tf.io.FixedLenFeature([], dtype=tf.string),
           'bur_vol': tf.io.FixedLenFeature([], dtype=tf.string)}})

    # All of the array preprocessor features are serialized integer arrays
    for key, val in preprocessor.tfrecord_features.items():
        if val.dtype == tf.string:
            parsed[key] = tf.io.parse_tensor(
                parsed[key], out_type=preprocessor.output_types[key])
    
    # Pop out the prediction target from the stored dictionary as a seperate dict
    parsed['spin'] = tf.io.parse_tensor(parsed['spin'], out_type=tf.float64)
    parsed['bur_vol'] = tf.io.parse_tensor(parsed['bur_vol'], out_type=tf.float64)
    
    spin = parsed.pop('spin')
    bur_vol = parsed.pop('bur_vol')
    targets = {'atom_inf_mask': spin, 'bur_vol': bur_vol}
    
    return parsed, targets


# Here, we have to add the prediction target padding onto the input padding
padded_shapes = (preprocessor.padded_shapes(max_atoms=None, max_bonds=None),
                 {'atom_inf_mask': [None], 'bur_vol': [None]})

padding_values = (preprocessor.padding_values,
                  {'atom_inf_mask': tf.constant(np.nan, dtype=tf.float64),
                   'bur_vol': tf.constant(np.nan, dtype=tf.float64)})

num_train = len(np.load('redf_split.npz', allow_pickle=True)['train'])
batch_size = 128
atom_features = 128
num_messages = 6

train_dataset = tf.data.TFRecordDataset('tfrecords_redf/train.tfrecord.gz', compression_type='GZIP')\
    .map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .cache().shuffle(buffer_size=num_train).repeat()\
    .padded_batch(batch_size=batch_size,
                  padded_shapes=padded_shapes,
                  padding_values=padding_values)\
    .prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset('tfrecords_redf/valid.tfrecord.gz', compression_type='GZIP')\
    .map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .cache().shuffle(buffer_size=500).repeat()\
    .padded_batch(batch_size=batch_size,
                  padded_shapes=padded_shapes,
                  padding_values=padding_values)\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
from tensorflow.keras import layers

spin_prediction = model.output
#spin_prediction._name = 'spin'

atom_state = model.layers[-5].output
bur_vol_bias =  layers.Embedding(preprocessor.atom_classes, 1,
                                 name='bur_vol_bias', mask_zero=True)(model.inputs[0])
atom_state = layers.Dense(1, name='bur_vol_dense')(atom_state)
bur_vol_prediction = layers.Add(name='bur_vol')([atom_state, bur_vol_bias])

In [7]:
model = tf.keras.Model(model.inputs, [spin_prediction, bur_vol_prediction])
learning_rate = tf.keras.optimizers.schedules.InverseTimeDecay(1E-4, 1, 1E-5)
model.compile(loss={'atom_inf_mask': KLWithLogits(), 'bur_vol': nfp.masked_mean_absolute_error},
              loss_weights={'atom_inf_mask': 1, 'bur_vol': 1E-3},
              optimizer=tf.keras.optimizers.Adam(learning_rate))

In [8]:
model.fit(train_dataset,
          validation_data=valid_dataset,
          steps_per_epoch=math.ceil(num_train/batch_size),
          validation_steps=math.ceil(5000/batch_size),
          epochs=10,
          verbose=1)

Epoch 1/10
107/107 [==============================] - 5s 43ms/step - loss: 0.0528 - atom_inf_mask_loss: 0.0372 - bur_vol_loss: 15.5152 - val_loss: 0.0417 - val_atom_inf_mask_loss: 0.0363 - val_bur_vol_loss: 5.4085
Epoch 2/10
107/107 [==============================] - 3s 29ms/step - loss: 0.0358 - atom_inf_mask_loss: 0.0313 - bur_vol_loss: 4.4545 - val_loss: 0.0401 - val_atom_inf_mask_loss: 0.0363 - val_bur_vol_loss: 3.7623
Epoch 3/10
107/107 [==============================] - 3s 28ms/step - loss: 0.0320 - atom_inf_mask_loss: 0.0286 - bur_vol_loss: 3.4102 - val_loss: 0.0392 - val_atom_inf_mask_loss: 0.0362 - val_bur_vol_loss: 3.0671
Epoch 4/10
107/107 [==============================] - 3s 28ms/step - loss: 0.0292 - atom_inf_mask_loss: 0.0263 - bur_vol_loss: 2.9010 - val_loss: 0.0363 - val_atom_inf_mask_loss: 0.0336 - val_bur_vol_loss: 2.7534
Epoch 5/10
107/107 [==============================] - 3s 28ms/step - loss: 0.0266 - atom_inf_mask_loss: 0.0239 - bur_vol_loss: 2.6235 - val_loss: 0